In [10]:
import json
import os
from library_dicom.dicom_processor.tools.folders import *
from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *
import numpy as np
import SimpleITK as sitk
import csv
import shutil

In [ ]:
#Script to write csv to use in Deep Learning 

In [ ]:
#JSON dataset 

In [11]:
json_path = '/media/oncopole/83c5223d-7a01-4ed0-b268-b877a7da96e2/REMARC_Validated_DICOMS/validated_dataset.json'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)

print(len(dataset))

182


In [12]:
#Liste nifti 

nifti_directory = '/media/oncopole/83c5223d-7a01-4ed0-b268-b877a7da96e2/REMARC_NIFTI'
liste = os.listdir(nifti_directory)
liste_nifti = []
for nifti in liste : 
    liste_nifti.append(os.path.join(nifti_directory, nifti))

In [13]:
dataset[0]

['/media/oncopole/83c5223d-7a01-4ed0-b268-b877a7da96e2/REMARC_Validated_DICOMS/11009101226011/pet0/1.3.46.670589.28.2.10.4.9167.3403.2.2520.0.1282553133/1.3.46.670589.28.2.10.4.9167.3403.2.2520.0.1282553133',
 'PT',
 '/media/oncopole/83c5223d-7a01-4ed0-b268-b877a7da96e2/REMARC_Validated_DICOMS/11009101226011/pet0/1.2.840.113704.1.111.1140.1282550706.9/1.2.840.113704.1.111.1140.1282550706.9',
 'CT',
 '2.16.840.1.113669.632.20.210791.10000113172',
 'V0',
 '11009101226011',
 '/media/oncopole/DD 2To/REMARC_csv/11009101226011_aug 23_2010.csvASC.csv']

In [14]:
for data in dataset : 
    uid = data[4]
    subliste = []
    for nifti in liste_nifti : 
        if uid in nifti : 
            subliste.append(nifti)
            subliste.sort()
            
    for i in range(4):
        data.append(subliste[i])
    

In [15]:
data

['/media/oncopole/83c5223d-7a01-4ed0-b268-b877a7da96e2/REMARC_Validated_DICOMS/11009101036021/pet0/1.2.840.113704.7.32.0.28.2.10.48.18553.24906.2.3564.0.1292924837/1.2.840.113704.7.32.0.28.2.10.48.18553.24906.2.3564.0.1292924837',
 'PT',
 '/media/oncopole/83c5223d-7a01-4ed0-b268-b877a7da96e2/REMARC_Validated_DICOMS/11009101036021/pet0/1.2.840.113704.7.32.1.2.840.113704.1.111.3260.1292923183.8/1.2.840.113704.7.32.1.2.840.113704.1.111.3260.1292923183.8',
 'CT',
 '1.2.840.113704.1.111.1460.1292922720.9',
 'V0',
 '11009101036021',
 '/media/oncopole/DD 2To/REMARC_csv/11009101036021_dec 21_2010.csvLV1.csv',
 []]

In [ ]:
#Check if CT PET MASK MERGED for pet0 

In [ ]:
for serie in dataset : 
    if len(serie) != 12 : 
        print(serie)

In [ ]:
print("nombre de study : ", len(dataset))

In [ ]:
#create folder to save nifti

for serie in dataset: 
    if 'V0' in serie : 
        #patient_name = serie[0]
        patient_id = serie[6]
        pet_type = "pet0"
        nifti_ct = serie[8]
        nifti_pet = serie[9]
        nifti_mask = serie[10] #if pet0
        nifti_merged = serie[11]

        folder = '/media/deeplearning/Elements/AHL_NIFTI'+'/'+patient_id

        if not os.path.exists(folder):
            os.makedirs(folder)
            print(folder, 'created')
        else : print(folder, "Dossier patient déjà crée")

        subfolder = folder+'/'+pet_type
        if not os.path.exists(subfolder) :
            os.makedirs(subfolder)
            print(subfolder, 'created')
        else : print(subfolder, "Déjà creer => already folder for ", pet_type)

        ct_path = nifti_ct
        pt_path = nifti_pet
        mask_path = nifti_mask
        merged_path = nifti_merged

        new_ct_path = subfolder+'/'+nifti_ct.split('/')[-1]
        new_pet_path = subfolder+'/'+nifti_pet.split('/')[-1]
        new_mask_path = subfolder+'/'+nifti_mask.split('/')[-1]
        new_merged_path = subfolder+'/'+nifti_merged.split('/')[-1]
        try : 

            #os.rename(ct_path, new_ct_path)
            #os.rename(pt_path, new_pet_path)
            #os.rename(mask_path, new_mask_path)
            #os.rename(merged_path, new_merged_path)

            serie[8] = new_ct_path
            serie[9] = new_pet_path 
            serie[10] = new_mask_path
            serie[11] = new_merged_path

        except Exception as err : 
            print(err)

            
    else : 
        patient_name = serie[0]
        patient_id = serie[1]
        pet_type = serie[3]
        nifti_ct = serie[7]
        nifti_pet = serie[8]

        folder = '/media/deeplearning/Elements/AHL_NIFTI'+'/'+patient_id

        if not os.path.exists(folder):
            os.makedirs(folder)
            print(folder, 'created')
        else : print(folder, "Dossier patient déjà crée")

        subfolder = folder+'/'+pet_type
        if not os.path.exists(subfolder) :
            os.makedirs(subfolder)
            print(subfolder, 'created')
        else : print(subfolder, "Déjà creer => already folder for ", pet_type)

        ct_path = nifti_ct
        pt_path = nifti_pet

        new_ct_path = subfolder+'/'+nifti_ct.split('/')[-1]
        new_pet_path = subfolder+'/'+nifti_pet.split('/')[-1]

        try : 
            os.rename(ct_path, new_ct_path)
            os.rename(pt_path, new_pet_path)

            serie[7] = new_ct_path
            serie[8] = new_pet_path 

        except Exception as err : 
            print(err)

In [ ]:
nifti_directory = '/media/deeplearning/Elements/AHL_NIFTI'
filename = 'AHL2011_NIFTI_V4.csv'

with open(os.path.join(nifti_directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT NAME", "PATIENT ID", "STUDY UID", "PET", "NIFTI_CT", "NIFTI_PET", "NIFTI_MASK", "NIFTI_MERGED"])
    for serie in data : 
        csv_writer.writerow(["", serie[6], serie[4], 'pet0', serie[7], serie[8], serie[9]])
